In [12]:
import pandas as pd

df = pd.read_csv("../data/real_final.csv")

In [13]:
print(df.isnull().sum())

GameDate              0
home_Team             0
away_Team             0
Result                0
home_Recent_5_Win     0
home_Recent_5_Loss    0
home_OPS              0
home_BB               0
home_HBP              0
home_SO               0
home_ERA              0
home_BB_p             0
home_K                0
home_IR               0
home_IS               0
home_TBF              0
away_OPS              0
away_BB               0
away_HBP              0
away_SO               0
away_ERA              0
away_BB_p             0
away_K                0
away_IR               0
away_IS               0
away_TBF              0
dtype: int64


결측치가 없으므로 바로 진행

In [14]:
# feature와 label 나누기
df = df.drop(columns=['GameDate'])
X = df.drop(columns=['Result']) 
y = df['Result']

In [15]:
# 원-핫 인코딩 (prefix로 home/away 구분!)
team_encoded = pd.get_dummies(
    X[['home_Team', 'away_Team']].reset_index(drop=True),
    columns=['home_Team', 'away_Team'],
    prefix=['home_Team', 'away_Team']
).astype(int)

X_numeric = X.drop(columns=['home_Team', 'away_Team']).reset_index(drop=True)
X_scaled = pd.concat([X_numeric, team_encoded], axis=1)

In [16]:
from sklearn.model_selection import train_test_split

# 80% 학습, 20% 테스트로 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

In [8]:
pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB 435.7 kB/s eta 0:05:45
   ---------------------------------------- 0.2/150.0 MB 2.0 MB/s eta 0:01:17
   ---------------------------------------- 0.4/150.0 MB 2.8 MB/s eta 0:00:53
   ---------------------------------------- 0.5/150.0 MB 2.5 MB/s eta 0:01:00
   ---------------------------------------- 0.6/150.0 MB 2.6 MB/s eta 0:00:59
   ---------------------------------------- 0.6/150.0 MB 2.6 MB/s eta 0:00:59
   ---------------------------------------- 0.6/150.0 MB 2.6 MB/s eta 0:00:59
   ---------------------------------------- 0.9/150.0 MB 2.5 MB/s eta 0:01:00
   ---------------------------------------- 1.0/150.0 MB 2.5 MB/s eta 0:00:59
   ---------------------------------------- 1.0/150.0 MB 2.5 MB/s eta 0:00:59
   ---------------------------------------- 1.2/150.0 MB 2.4 MB/s eta 0:01:0

In [24]:
pip install --upgrade xgboost

Note: you may need to restart the kernel to use updated packages.


In [17]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=100,       # 트리 개수
    max_depth=4,            # 트리 최대 깊이
    learning_rate=0.1,      # 학습률
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'   # 이진 분류일 때 기본 로그 손실 사용
)

model.fit(X_train, y_train)

c:\Users\cjlee\anaconda3\envs\pandas\lib\site-packages\xgboost\training.py:183: UserWarning: [14:48:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [18]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]  # 클래스 1(홈팀 승)일 확률

In [19]:
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ AUC Score:", roc_auc_score(y_test, y_proba))
print("\n✅ Classification Report:\n", classification_report(y_test, y_pred))
print("✅ Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

✅ Accuracy: 0.8754098360655738
✅ AUC Score: 0.9477309911306295

✅ Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.86      0.87       294
           1       0.88      0.89      0.88       316

    accuracy                           0.88       610
   macro avg       0.88      0.88      0.88       610
weighted avg       0.88      0.88      0.88       610

✅ Confusion Matrix:
 [[254  40]
 [ 36 280]]


In [21]:
# 훈련 세트 예측
y_train_pred = best_model.predict(X_train)
y_train_proba = best_model.predict_proba(X_train)[:, 1]

# 테스트 세트 예측
y_test_pred = best_model.predict(X_test)
y_test_proba = best_model.predict_proba(X_test)[:, 1]

# 정확도 비교
print("📘 Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("📙 Test Accuracy :", accuracy_score(y_test, y_test_pred))

# AUC 비교
print("📘 Train AUC:", roc_auc_score(y_train, y_train_proba))
print("📙 Test AUC :", roc_auc_score(y_test, y_test_proba))


📘 Train Accuracy: 0.9794997949979499
📙 Test Accuracy : 0.8704918032786885
📘 Train AUC: 0.9980534608133584
📙 Test AUC : 0.9447494187548436


✅ 전체적인 모델 평가


🎯 1. 정확도(Accuracy)

Train Accuracy: 0.979 → 거의 완벽하게 학습 데이터를 분류

Test Accuracy: 0.875 → 실제 데이터에서도 매우 높은 예측 성능

▶️ 정확도 차이(10%↓)는 과적합을 시사하지만 여전히 테스트 성능은 매우 뛰어남


🎯 2. AUC (Area Under Curve)

Train AUC: 0.998

Test AUC: 0.947

▶️ 0.94 이상은 매우 좋은 수준.

즉, 모델이 홈팀 승/패를 강하게 구분할 수 있는 능력을 가지고 있음.


🎯 3. Precision, Recall, F1-Score

클래스	precision	recall	f1-score

홈팀 패 (0)	0.88	0.86	0.87

홈팀 승 (1)	0.88	0.89	0.88

정확도와 균형이 잘 잡힌 precision/recall: 모델이 한 쪽으로 치우치지 않음

클래스 간 성능 균형이 좋음: 홈팀 승/패 모두 잘 예측하고 있음


🎯 4. Confusion Matrix 분석

[[254  40]  → 실제 0인데 1로 잘못 예측: 40건

 [ 36 280]] → 실제 1인데 0으로 잘못 예측: 36건

오류 비율이 대칭적이고, 전체적으로 예측 정확도가 매우 안정적임을 보여줌


⚖️ 최종 평가

항목	평가 내용

🎯 예측 정확도	⭐ 매우 높음 (87% 이상)

🎯 일반화(AUC) 성능	⭐ 매우 높음 (0.94+)

🧠 과적합 위험	⚠️ 일부 존재하지만 실제 성능 저하 적음

⚖️ 클래스 균형	⭐ 잘 맞음 (precision/recall 균형)

🧪 실전 적용 가능성	✅ 신뢰도 높은 예측 가능, 응용 가능

✅ 1. 과적합 완화 (Overfitting 완화)

🎯 목적:

훈련 데이터에 너무 딱 맞춰진 모델을 일반화시켜 테스트/실전에서도 안정적이게 하기

🔧 방법:

기법	설명

max_depth 감소	트리 깊이를 줄이면 모델 복잡도가 줄고, 과적합 방지

subsample < 1.0	학습 샘플의 일부만 사용해서 일반화 성능 향상

colsample_bytree < 1.0	일부 feature만 사용

early_stopping_rounds 사용	검증 성능이 개선되지 않으면 학습 중단